In [ ]:
#pip install opencv-python
import sys
import os
os.sys.path
import cv2
import numpy as np


from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#Tensorflow keras for CNN model
#pip install tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
folder_dir = "flowers"

label=[]
data=[]

SIZE = 128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir,folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir,folder,file))
            img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) # converts
            im = cv2.resize(img_rgb,(SIZE,SIZE))
            data.append(im)
        else:
            continue
 


In [ ]:
data_array = np.array(data)
label_array = np.array(label)

In [ ]:

#print(label_arr)

encoder =LabelEncoder()
y = encoder.fit_transform(label)

#print(y)
y =  to_categorical(y,5) # 5 flower type -converts all 5 into categorical
#print(y)
#x = data_arr/255
#x =[]
#for im in data:
 #   x.append(cv2.normalize(im, None, 0, 255,cv2.NORM_MINMAX, dtype=cv2.CV_32F))
x = data_array/255


In [ ]:
print(np.shape(y))
print(np.shape(data))

In [ ]:
#show normalized image
cv2.imshow('k',x[0])
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
X_train, X_test, Y_train, Y_test =  train_test_split(x, y, test_size=0.20, random_state=10)

In [ ]:
print(len(X_train))
print(len(Y_train))

In [ ]:
model = Sequential()
#input
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation='relu', input_shape = (SIZE,SIZE,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
#hidden
model.add(Conv2D(filters = 128, kernel_size=(3,3),padding = 'Same',activation='relu'))
model.add(Conv2D(filters = 128, kernel_size=(3,3),padding = 'Same',activation='relu'))
model.add(Conv2D(filters = 128, kernel_size=(3,3),padding = 'Same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#output
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(5, activation = "softmax"))

In [ ]:

datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range = 0.20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        horizontal_flip=True,
        vertical_flip=True)

datagen.fit(X_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size=32
epochs=64
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,Y_test),
                              verbose = 1)

In [ ]:
categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25, 40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(Y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(Y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(Y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])], color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE, SIZE, 3), cmap='gray')

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("TensorFlow **IS** using the GPU")
else:
  print("TensorFlow **IS NOT** using the GPU")

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))